<a href="https://colab.research.google.com/github/matardy/Customer-support-chatbot/blob/main/notebook/FineTunning_GPT2_OnBankOfAmericaCorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gpt-2-simple
!pip install datasets
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00


# Preprocesamiento de Datos y Generacion de Datos para el fine tunning

In [2]:
from datasets import load_dataset
ds = load_dataset("argilla/llama-2-banking-fine-tune",split="train")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['request', 'response-1', 'response-2', 'preference', 'preference-suggestion', 'preference-suggestion-metadata', 'correct-response', 'correct-response-suggestion', 'correct-response-suggestion-metadata', 'external_id', 'metadata'],
    num_rows: 100
})

In [ ]:
ds[0]

{'request': 'I tried to make a transfer but it failed',
 'response-1': " Thank you for choosing the Bank of America for your financial needs! I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? This will help me better understand the issue and assist you in resolving it as quickly as possible. Additionally, have you tried checking your account balance or recent transactions to see if the transfer has been processed? If you have any further questions or concerns, please don't hesitate to ask. Thank you for trusting the Bank of America with your financial matters!",
 'response-2': " Hello there! Thank you for choosing the Bank of America for your financial needs. I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? Additionally, do you

In [ ]:
ds[0]['request']

'I tried to make a transfer but it failed'

In [ ]:
ds[0]['response-1']

" Thank you for choosing the Bank of America for your financial needs! I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? This will help me better understand the issue and assist you in resolving it as quickly as possible. Additionally, have you tried checking your account balance or recent transactions to see if the transfer has been processed? If you have any further questions or concerns, please don't hesitate to ask. Thank you for trusting the Bank of America with your financial matters!"

In [3]:
with open("train_dataset.txt", "w") as f:
    for i, ds in enumerate(ds):
        request = ds.get('request', 'N/A')  # Obtiene el valor de 'request' o 'N/A' si no está presente
        response_1 = ds.get('response-1', 'N/A')  # Obtiene el valor de 'response-1' o 'N/A' si no está presente
        f.write("<|startoftext|>")
        f.write(f"User: {request}\n")
        f.write(f"Customer Service: {response_1}\n")
        f.write("<|endoftext|>")
        f.write("\n")  # Espacio entre pares de preguntas y respuestas

In [ ]:
!nvidia-smi

Thu Sep  7 03:47:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first.

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [4]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 3.93Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:01, 563kit/s]
Fetching hparams.json: 1.05Mit [00:00, 4.32Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:02, 7.97Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 3.55Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:01, 814kit/s]
Fetching vocab.bpe: 1.05Mit [00:01, 847kit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [3]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [6]:
file_name = "train_dataset.txt"

In [10]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies.

In [11]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=140, # se encontro el mejor valor para el dataset.
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              #sample_every=200,
              save_every=50,

              )

Loading checkpoint models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  9.62it/s]

dataset has 19653 tokens
Training...


[10 | 28.08] loss=1.02 avg=1.02
[20 | 49.58] loss=0.73 avg=0.87
[30 | 71.50] loss=0.64 avg=0.79
[40 | 93.84] loss=0.35 avg=0.68
[50 | 116.00] loss=0.19 avg=0.58
Saving checkpoint/run1/model-50
[60 | 145.62] loss=0.15 avg=0.51
[70 | 167.91] loss=0.10 avg=0.45
[80 | 190.08] loss=0.08 avg=0.40
[90 | 212.08] loss=0.07 avg=0.36
[100 | 234.10] loss=0.06 avg=0.33
Saving checkpoint/run1/model-100


Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
: Any time you don't see anything, do you have something to report? Sometimes you may also get an error message that says "It's not possible to make a statement on your card until we receive it."
Great news, folks! We're here to help you with your issue. Bankersief works by tracking your transactions and updating your account information. Once a while, though, something goes wrong with your account and your account balance may go up or down. Sometimes, this can happen after a while, but only if you made a payment and noticed a change in the account information.
If you noticed any such transactions on your account before today, you can try contacting our customer service team to inquire about them. We're here to help you resolve the issue as quickly as possible.
Please keep in mind that we take any errors or discrepancies on your account very seriously and will work to resolve them as quickly as possible. If there was an error in charging you a fee, we will wo

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [12]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [4]:
# copiar el modelo de drive a colab
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-140


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [6]:
gpt2.generate(sess, run_name='run1')

Please note that these limits may vary depending on your account type and the payment method you're using. If you have any further questions or concerns, please feel free to ask!
<|endofext|>
<|startoftext|>User: Which currencies do I need to add ASAP?
Customer Service:  Hello there! Thank you for choosing the Bank of America for your financial needs. I'm happy to help you with any questions you may have. To answer your question, the next time you add a new currency, be sure to check with us at the bank to see if we're happy to add you to our account. If you're looking to add a new currency, please refer to our account guidelines for assistance.
If you're looking to add a new currency, please contact our customer service team at 1-800-432-1000. Our representatives are available 24/7 to assist you. Thank you for trusting the Bank of America with your financial needs.
Thank you for choosing the Bank of America, and please let me know if there's anything else I can help you with.
<|endofe

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [23]:
prompt = """User: Hey my card was stolen
Customer Service: Hello there! Thank you for reaching out to us about the theft of your card. I'm here to help you resolve the issue. Can you please provide me with more details about the transaction? For example, what was the amount of the transaction, and when did you make the deposit? Additionally, have you checked your account activity to see if the transaction has been processed? If you've already verified that your account has been processed, please let me know, and we can work on resolving the issue together. Thank you for trusting the Bank of America with your banking needs.
User: I made that deposit yesterday.
Customer Service: """

response = gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix=prompt,
              top_k = 1,
              top_p=0.9,
              include_prefix = True,
              truncate = '<|endofext|>',
              return_as_list=True
              )[0]



User: Hey my card was stolen 
 Customer Service: Hello there! Thank you for reaching out to us about the theft of your card. I'm here to help you resolve the issue. Can you please provide me with more details about the transaction? For example, what was the amount of the transaction, and when did you make the deposit? Additionally, have you checked your account activity to see if the transaction has been processed? If you've already verified that your account has been processed, please let me know, and we can work on resolving the issue together. Thank you for trusting the Bank of America with your banking needs.
User: I made that deposit yesterday.
Customer Service:  Thank you for trusting Bank of America with your financial needs! I'm happy to help you with your question.
To answer your question, we do not account for any errors or discrepancies in our deposits. Our standard procedure is to update our account information or contact our bank directly. However, please note that errors 